In [24]:
from numpy.random import normal
import numpy as np
from numpy.linalg import inv
from sympy import symbols, diff

import numdifftools as nd
def partial_function(f___,input,pos,value):
    tmp  = input[pos]
    input[pos] = value
    ret = f___(*input)
    input[pos] = tmp
    return ret
def partial_derivative(f,input):
    ret = np.empty(len(input))
    for i in range(len(input)):
        fg = lambda x:partial_function(f,input,i,x)
        ret[i] = nd.Derivative(fg)(input[i])
    return ret

In [66]:
#Create the model
T = 10

#Model 3: non-linear model
#x_k+1 = f(k,x_k) + gaussian noise (mean 0,variance 1)
#y_k = h(k,x_k) + gaussian noise  (mean 0,variance 0.1)
# f = lambda k,x_k: 2*x_k + 1/x_k + k  
f = lambda k,x_k: (k+1)*x_k  
h = lambda k,x_k: (1 + 1/(k+1))*x_k
B = np.zeros([1,1])
D = np.zeros([1,1])
Q = np.identity(1) * 5
R = np.identity(1) * 5
mean = np.zeros([1,1])

x = np.zeros([T,1])
y = np.zeros([T,1])
u = np.zeros([T,1])

#Initialization
# for k in range(T-1):
#     u[k] = 0
x[0] = normal(0,1)

#Propagation
for k in range(T-1):
    x[k+1] = f(k,x[k]) + np.dot(B,u[k]) + normal(mean,Q)
for k in range(T):
    y[k] = h(k,x[k])  + np.dot(D,u[k]) + normal(mean,R)

print(x)
# print(y)
# print(C_all)

[[ -5.29866326e-01]
 [  6.12551335e+00]
 [  1.71184505e+01]
 [  3.72103595e+01]
 [  7.77103801e+01]
 [  1.58999386e+02]
 [  3.22698538e+02]
 [  6.46121684e+02]
 [  1.29451486e+03]
 [  2.59808597e+03]]


In [96]:
#Extented Kalman filter
#from page 25 of ftp://icf.org.ru/pub/docs/linux-support/computer%20science/Artificial%20Intelligence/Neural%20networks/Kalman%20Filtering%20and%20Neural%20Networks%20-%20Simon%20Haykin.pdf

#Initialization
V = np.zeros([T,1])
V[0] = np.identity(1)
x_f = np.zeros([T,1])
# x_f[0] = normal(0,V[0])
x_f[0] = 1.
V_plus_all = np.zeros([T,1])
A_all = np.zeros([T,1])
C_all = np.zeros([T,1])
x_plus_all = np.zeros([T,1])

#Propagation
for k in range(1,T):
#     print("k= ",k)
    A_all[k-1] = np.matrix(partial_derivative(f,np.array([k-1,x_f[k-1]]))[1]) #derivative of f, with respect to x_k
    x_plus = f(k-1,x_f[k-1])
    x_plus_all[k] = x_plus
    C_all[k] = np.matrix(partial_derivative(h,np.array([k,x_plus]))[1]) #derivative of h, with respect to x_k
    V_plus = np.matrix(A_all[k-1].dot(V[k-1])).dot(A_all[k-1].T) + Q 
    V_plus_all[k] = V_plus
    
#     print(C_all[k].dot(V_plus.dot(C_all[k].T) + R))
    K = V_plus.dot(C_all[k].T).dot(inv(C_all[k].dot(V_plus.dot(C_all[k].T) + R)))
    x_f[k] = x_plus + K.dot(y[k] - h(k,x_plus))
    V[k] = (np.identity(K.shape[0]) - K.dot(C_all[k])).dot(V_plus)

#     print("x_plus",x_plus)
#     print("h",h(k,x_plus))
# #     print("A_all",A_all)
# #     print("C_all",C_all)
#     print("V_plus",V_plus)
#     print("K",K)
#     print("x_f[k]",x_f[k])
#     print("V[k]",V[k])
#     print()

A_all[T-1] = np.matrix(partial_derivative(f,np.array([T-1,x_f[T-1]]))[1]) #derivative of f, with respect to x_k
print(x_f)
print('\n')
print(x_f - x)

[[  1.00000000e+00]
 [  3.21442469e+00]
 [  1.59537391e+01]
 [  3.69934206e+01]
 [  8.07234948e+01]
 [  1.63036911e+02]
 [  3.24657433e+02]
 [  6.44814364e+02]
 [  1.29164993e+03]
 [  2.60053432e+03]]


[[ 1.52986633]
 [-2.91108866]
 [-1.16471144]
 [-0.21693882]
 [ 3.01311474]
 [ 4.03752575]
 [ 1.95889554]
 [-1.30732018]
 [-2.86493192]
 [ 2.44835518]]


/home/vincent/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: RuntimeWarning: divide by zero encountered in double_scalars
/home/vincent/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: RuntimeWarning: divide by zero encountered in double_scalars


In [98]:
#Recursive
#F = A, B = H, P = V

#Initialization
x_b = np.zeros([T,1])
x_b[T-1] = x_f[-1]
V_b = np.zeros([T,1])
V_b[-1] = V[-1]
# print(len(x_f))
# print(len(A_all))
print(len(x_plus_all))

#Propagation
#Remark: compared to the table 1.2 on page 17, we changed all the indexes by -1
for k in range(T-1, 0, -1): #from T-1 to 1 by -1 increments
        
    A_b = np.matrix(V_plus_all[k-1].dot(A_all[k].T)).dot(inv(np.matrix(V[k])))
    V_b[k-1] = V_plus_all[k-1] - A_b.dot(V[k] - V_b[k]).dot(A_b.T)
#     x_b[k-1] = x_f[k-1] + A_b.dot(x_b[k] - x_f[k])
    x_b[k-1] = x_f[k-1] + A_b.dot(x_b[k] - x_plus_all[k])
    
    print(A_b)
    print(A_b.dot(x_b[k] - x_f[k]))
    print(x_b[k-1])
    
print()
print(x_b)  
print('\n')
print(x_f - x_b)
print()
print("after forward pass, we get",'\n',x_f - x)
print()
print("after backward pass, we get",'\n',x_b - x)

10
[[ 10.53450416]]
[[ 0.]]
[ 1388.92235174]
[[ 10.51608104]]
[[ 1022.92471808]]
[ 1615.36529143]
[[ 10.48344837]]
[[ 10174.72053949]]
[ 10389.26420204]
[[ 10.42064871]]
[[ 104879.73156809]]
[ 104975.8416229]
[[ 10.27295354]]
[[ 1076737.07272495]]
[ 1076792.91033668]
[[ 9.79364183]]
[[ 10544933.51267364]]
[ 10545006.83680038]
[[ 7.9205034]]
[[ 83521469.48760392]]
[ 83521509.38709435]
[[ 4.13216227]]
[[  3.45124364e+08]]
[  3.45124401e+08]
[[ 0.]]
[[ 0.]]
[ 1.]

[[  1.00000000e+00]
 [  3.45124401e+08]
 [  8.35215094e+07]
 [  1.05450068e+07]
 [  1.07679291e+06]
 [  1.04975842e+05]
 [  1.03892642e+04]
 [  1.61536529e+03]
 [  1.38892235e+03]
 [  2.60053432e+03]]


[[  0.00000000e+00]
 [ -3.45124398e+08]
 [ -8.35214934e+07]
 [ -1.05449698e+07]
 [ -1.07671219e+06]
 [ -1.04812805e+05]
 [ -1.00646068e+04]
 [ -9.70550928e+02]
 [ -9.72724263e+01]
 [  0.00000000e+00]]

after forward pass, we get 
 [[ 1.52986633]
 [-2.91108866]
 [-1.16471144]
 [-0.21693882]
 [ 3.01311474]
 [ 4.03752575]
 [ 1.95889